Nutuk RAG w/ LangChain, Gemini, Chroma

In [1]:
# Install libraries LangChain, Chroma, GoogleGenAI etc.

In [2]:
from dotenv import load_dotenv
import os


load_dotenv()

True

## Import the Nutuk PDF

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("nutuk.pdf")
data = loader.load()

len(data)

430

## Verileri Parçalara Ayırma (Chunking)

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Chunk_size 1000 olarak ayarladım, her parçada maksimum 1000 karakter olacak.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print(f"Toplam parça sayısı: {len(docs)}")

Toplam parça sayısı: 2042


In [5]:
docs[7]

Document(metadata={'producer': 'Foxit G SDK 1.1 - Foxit Corporation', 'creator': 'PyPDF', 'creationdate': 'D:20141021111813', 'source': 'nutuk.pdf', 'total_pages': 430, 'page': 1, 'page_label': '2'}, page_content="etmek, savaş durumunun Doğu illerinde yarattığı yıkım ve yoksulluğa, hükûmet nezdinde\nteşebbüslerde bulunarak elden geldiğince çare aramaktan ibaretti.\nİstanbul'daki yönetim merkezinden verilmiş olan bu direktife uygun olarak, Erzurum şubesi, Doğu\nillerinde Türk'ün haklarını korumakla birlikte, Ermeni göçü sırasında görülen kötü davranışlarla\nhalkın hiçbir ilgisi bulunmadığını, Ermeni mallarının Rus istilâsına kadar korunduğunu, buna\nkarşılık müslümanlara pek gaddarca davranıldığını; hattâ verilen emre aykırı olarak, göçten\nalıkonan bazı Ermenilerin koruyucularına karşı yaptıkları kötülükleri, güvenilir belgelerle\nmedeniyet dünyasına duyurmaya ve Doğu illerine dikilmiş olan hırs yüklü bakışları hükümsüz\nbırakacak çalışmalar yapmaya karar veriyor (Erzurum şubesinin bas

## Embedding Oluşturma ve ChromaDB Kaydetme

In [6]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [7]:
# Embedding oluşturma

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

**ChromaDB vector database oluşturma**

In [8]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

**En yakın 10 chunk retrieve ediyor**

In [9]:
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 10}
)

In [18]:
retrieved_docs = retriever.invoke("Sakarya Meydan Muharebesi")

In [19]:
len(retrieved_docs)

10

In [20]:
print(retrieved_docs[0].page_content)

kudretinden yoksun birduruma getirdi.
Muharebe durumunun bu safhasını sezer sezmez hemen özelliklesağ kanadımızla Sakarya
ırmağı doğusunda düşman ordusunun sol kanadına ve daha sonra cephenin önemli yerlerinde
karşı taarruza geçtik.Yunan ordusu yenildi ve geri çekilmeye mecbur oldu.13 Eylül 1921
günüSakarya ırmağının doğusunda düşman ordusundan eser kalmadı. Böylece23 Ağustos
gününden 13 Eylül gününe kadar, bu günler de dahil olmaküzere, yirmi iki gün yirmi iki gece
aralıksız devam eden büyük ve kanlıSakarya Meydan Muharebesi yeni Türk devletinin tarihine,
dünyatarihinde pek az rastlanan büyük bir meydan muharebesi örneği kaydetti.
Saygıdeğer Efendiler, Başkomutanlık görevini fiilen üzerime aldığım zaman, Meclis'e ve millete
mutlaka başaracağımız yolundaki kesininancımı arz ve ilân etmekle ve bu inancımı, varlığımın
bütün haysiyetiniortaya atarak gerçekleştirmekle ilk manevî görevimi yapmış olduğumusanırım.


## Google Gemini API ile LLM entegrasyonu

tempature değerleri


* Düşük değerler (0.1-0.4): Daha keskin ve tutarlı cevaplar verir, model daha tahmin edilebilir hale gelir.
* Orta değerler (0.5-0.7): Hem daha mantıklı hem de yaratıcı cevaplar verebilir.
* Yüksek değerler (0.7-1): Daha rastgele, yaratıcılık daha yüksek ancak bu mantıksız cevaplar vermesine yol açabilir.


In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=500
)

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [23]:
system_prompt = (
    "Sen, Nutuk kitabındaki parçalara dayanarak soru-cevap görevlerini gerçekleştiren bir asistansın. "
    "Cevap verirken Gazi Mustafa Kemal Atatürk'ün hitap tarzını benimse: kararlı, tarihi temellere dayanan, milletine seslenir gibi. "
    "Cevapları, Nutuk'tan verilen bağlam parçalarına dayanarak ver. "
    "Eğer verilen bağlamda cevap yoksa, 'Bu konuda elimdeki vesikalar yetersizdir.' şeklinde cevapla. "
    "Yanıtın bir hitap gibi, veciz ve etkileyici olsun. En fazla üç cümle kullan."
    "\n\n"
    "{context}"
)



In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

## Soru - Cevap Zinciri Oluşturma (LLM + PROMPT)

In [26]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

## RAG Zinciri (RAG+LLM)

In [27]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

## Kullanıcı sorgusu ile cevap üretme

In [28]:
response = rag_chain.invoke({"input":"Samsun'a neden çıktınız?"})

In [29]:
print(response)

{'input': "Samsun'a neden çıktınız?", 'context': [Document(id='4b740686-e53e-412e-93b8-53d3d2cd079c', metadata={'creationdate': 'D:20141021111813', 'creator': 'PyPDF', 'page': 222, 'page_label': '223', 'producer': 'Foxit G SDK 1.1 - Foxit Corporation', 'source': 'nutuk.pdf', 'total_pages': 430}, page_content='Belki o tarihte, o bölgelerde bulunan sivil idare âmirleridir. Halk, gerçeği anlaranlamaz, derhal\nmilletin ortak isteğine katılmakta asla kararsızlık göstermemiştir.\nŞimdi Efendiler, yeniden inkılâbın tabiî sonuçlarından sayılan olaylardan bazılarına temas edelim\n:\nSAMSUN\'DAKİ SUBAYLAR ARASINDA SÖZDE PADİŞAH TARAFTARLIĞI VARMIŞ\n3\'ncü Kolordu Komutanı Selâhattin Bey\' den aldığım 29 Mart 1920 tarihli bir şifrede, "Sam sun\'da\nbulunan 15\' inci Tümen\'in maneviyatının bo zuk olduğundan ve sözde, subaylar arasında\nPadişahtaraftarlığı bulunduğundan" söz ediliyordu. "Subaylar, Padişâh aleyhindeverilecek emirleri\nyerine getirmeyeceklerini komutanlarına bildirmişler.Baskı yapıl

In [30]:
print(response['answer'])

Ben, bu görevin yerine getirilmesinin Samsun ve dolaylarındaki güvensizlik olaylarını yerinde görüp tedbir almak üzere Samsun'a kadar gitmek olduğunu düşünerek bu görevi kabul ettim.
